In [1]:
import digits
import tensorflow as tf
from time import time
import itertools as it
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
DATA_PATH = 'digits.csv'

LEARNING_RATE = 0.1
BATCH_SIZE = 32
EPOCHS = 30
MODEL_DIR = 'cnn-res/model_{}'

In [3]:
labels, pixels = digits.read(DATA_PATH)
pixels_sq = pixels.reshape(-1, 28, 28)

In [4]:
train_label, test_label, train_pixel, test_pixel = train_test_split(labels, pixels_sq, test_size=0.1, 
                                                                    stratify=labels, shuffle=True, random_state=0)
train_label, dev_label, train_pixel, dev_pixel = train_test_split(train_label, train_pixel, test_size=0.11111111, 
                                                                  stratify=train_label, shuffle=True, random_state=0)

In [5]:
train_label.shape, dev_label.shape, test_label.shape

((8000,), (1000,), (1000,))

In [6]:
train_pixel.shape, dev_pixel.shape, test_pixel.shape

((8000, 28, 28), (1000, 28, 28), (1000, 28, 28))

In [7]:
num_labels = len(set(labels))
num_labels

10

In [8]:
def model_fn(features, labels, mode):
    c1 = tf.layers.conv2d(features, 4, 3, activation=tf.nn.relu) # 26 x 26
    c2 = tf.layers.conv2d(c1, 4, 3, activation=tf.nn.relu) # 24 x 24
    m1 = tf.layers.max_pooling2d(c2, 2, 2) # 12 x 12
    c3 = tf.layers.conv2d(m1, 8, 3, activation=tf.nn.relu) # 10 x 10
    c4 = tf.layers.conv2d(c3, 8, 3, activation=tf.nn.relu) # 8 x 8
    c5 = tf.layers.conv2d(c4, 8, 3, activation=tf.nn.relu) # 6 x 6
    m2 = tf.layers.max_pooling2d(c5, 2, 2) # 3 x 3
    fc = tf.concat([tf.layers.flatten(m1),
                    tf.layers.flatten(m2)],
                   axis=-1)
    logits = tf.layers.dense(fc, num_labels)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        proba = tf.nn.softmax(logits, axis=-1)
        predictions = {
            'logits': logits,
            'probabilities': proba,
            'labels': tf.argmax(proba, axis=-1),
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.softmax_cross_entropy(labels, logits)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss)
    
    optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss, tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

In [9]:
def td_input_fn(pixels, labels, batch_size, epochs, is_training, buffer=100):
    ds = tf.data.Dataset.from_tensor_slices((pixels, labels))
    
    def preprocessing(p, lbl):
        p3d = tf.expand_dims(p, axis=-1)
        p3d = tf.to_float(p3d)
        p3d = tf.image.per_image_standardization(p3d)
        lbl_1h = tf.one_hot(lbl, num_labels, dtype=tf.float32)
        return p3d, lbl_1h
    
    ds = ds.map(preprocessing, num_parallel_calls=8)
    
    if is_training:
        ds = ds.shuffle(buffer * batch_size)
        
    ds = ds.batch(batch_size)
    ds = ds.repeat(epochs)
    ds = ds.prefetch(buffer)
    
    return ds

In [10]:
def pred_input_fn(pixels, batch_size, buffer=100):
    ds = tf.data.Dataset.from_tensor_slices(pixels)
    
    def preprocessing(p):
        p3d = tf.expand_dims(p, axis=-1)
        p3d = tf.to_float(p3d)
        p3d = tf.image.per_image_standardization(p3d)
        return p3d
        
    ds = ds.map(preprocessing, num_parallel_calls=8)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer)
    
    return ds

In [11]:
model_dir = MODEL_DIR.format(int(time()))
config = tf.estimator.RunConfig(tf_random_seed=0,
                                save_summary_steps=50,
                                save_checkpoints_steps=250,
                                keep_checkpoint_max=None)

clf = tf.estimator.Estimator(model_fn, model_dir, config)

INFO:tensorflow:Using config: {'_model_dir': 'cnn-res/model_1540252548', '_tf_random_seed': 0, '_save_summary_steps': 50, '_save_checkpoints_steps': 250, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001EF04EC5780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
train_input_fn = lambda: td_input_fn(train_pixel, train_label, BATCH_SIZE, EPOCHS, True)
dev_input_fn = lambda: td_input_fn(dev_pixel, dev_label, BATCH_SIZE, 1, False)

train_spec = tf.estimator.TrainSpec(train_input_fn)
eval_spec = tf.estimator.EvalSpec(dev_input_fn, None, start_delay_secs=0.1, throttle_secs=0.1)

In [13]:
tf.estimator.train_and_evaluate(clf, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 250 or save_checkpoints_secs None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into cnn-res/model_1540252548\model.ckpt.
INFO:tensorflow:cnn-res/model_1540252548\model.ckpt-0 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:loss = 2.3970046, step = 0
INFO:tensorflow:global_step/sec: 158.031
INFO:tensorflow:loss = 0.47077662, step = 100 (0.633 sec)
INFO:tensorflow:global_step/sec: 245.42
INFO:tensorflow:loss = 0.13946763, step = 200 (0.407 sec)
INFO:tensorflow:Saving checkpoints

INFO:tensorflow:Saving checkpoints for 2000 into cnn-res/model_1540252548\model.ckpt.
INFO:tensorflow:cnn-res/model_1540252548\model.ckpt-2000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-22-23:56:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-res/model_1540252548\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-22-23:56:13
INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 0.20862295
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: cnn-res/model_1540252548\model.ckpt-2000
INFO:tensorflow:global_step/sec: 75.8157
INFO:tensorflow:loss = 0.013140324, step = 2000 (1.303 sec)
INFO:tensorflow:global_step/sec: 263.931
INFO:tensorflow:loss = 0.04098838, step = 2100 (0.379 sec)
INFO

INFO:tensorflow:loss = 0.0019714946, step = 3800 (1.335 sec)
INFO:tensorflow:global_step/sec: 249.383
INFO:tensorflow:loss = 0.37306976, step = 3900 (0.401 sec)
INFO:tensorflow:Saving checkpoints for 4000 into cnn-res/model_1540252548\model.ckpt.
INFO:tensorflow:cnn-res/model_1540252548\model.ckpt-4000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-22-23:56:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-res/model_1540252548\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-22-23:56:29
INFO:tensorflow:Saving dict for global step 4000: global_step = 4000, loss = 0.24522942
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4000: cnn-res/model_1540252548\model.ckpt-4000
INFO:tensorflow:global_step/sec: 75.6367
INF

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5750: cnn-res/model_1540252548\model.ckpt-5750
INFO:tensorflow:global_step/sec: 79.1256
INFO:tensorflow:loss = 0.00026633494, step = 5800 (1.264 sec)
INFO:tensorflow:global_step/sec: 270.434
INFO:tensorflow:loss = 0.008970129, step = 5900 (0.370 sec)
INFO:tensorflow:Saving checkpoints for 6000 into cnn-res/model_1540252548\model.ckpt.
INFO:tensorflow:cnn-res/model_1540252548\model.ckpt-6000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-22-23:56:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-res/model_1540252548\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-22-23:56:45
INFO:tensorflow:Saving dict for global step 6000: global_step = 6000, loss = 0.48204452
I

({'loss': 0.24580304, 'global_step': 7500}, [])

In [ ]:
checkpoint_path = r'cnn-res/model_1540251725\model.ckpt-2750'

In [ ]:
test_input_fn = lambda: pred_input_fn(test_pixel, BATCH_SIZE)

test_pred = clf.predict(test_input_fn, predict_keys=['labels'], checkpoint_path=checkpoint_path)
test_pred = [p['labels'] for p in test_pred]
test_pred = np.asarray(test_pred)

In [ ]:
eval_input_fn = lambda: td_input_fn(test_pixel, test_label, BATCH_SIZE, 1, False)
clf.evaluate(eval_input_fn, checkpoint_path=checkpoint_path)

In [ ]:
print(classification_report(test_label, test_pred))

In [ ]:
cm = confusion_matrix(test_label, test_pred)
cm